In [1]:
import astropy.time
from lsst.daf.butler import Butler, Timespan

In [2]:
butler = Butler('/repo/embargo')
registry = butler.registry

In [3]:
scratch_butler = Butler('/home/j/jarugula/scratch', writeable=True)
scratch_registry = scratch_butler.registry

In [4]:
datasetType = ['raw', 'goodSeeingCoadd', 'calexp']
collections = ['LATISS/raw/all/', 'LATISS/runs/AUXTEL_DRP_IMAGING_2023-08ABC-07AB-05AB/w_2023_35/PREOPS-3726/20230912T151732Z', 
               'LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616']
dataId = {'band':'i'}

## run queryDatasetTypes with a list of datasettypes as input

In [5]:
dataset_dims = registry.queryDatasetTypes(datasetType)

In [6]:
dataset_dims

[DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure),
 DatasetType('goodSeeingCoadd', {band, skymap, tract, patch}, ExposureF),
 DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF)]

In [7]:
print(dataset_dims[0].dimensions)
print(dataset_dims[1].dimensions)

{band, instrument, detector, physical_filter, exposure}
{band, skymap, tract, patch}


In [8]:
dtype_exposure = []
dtype_ingest = []
for dtype in datasetType:
    if any(
        dim in ["exposure", "visit"]
        for dim in [
            d.name for d in registry.queryDatasetTypes(dtype)[0].dimensions
        ]
    ):
        dtype_exposure.append(dtype)
    else:
        dtype_ingest.append(dtype)

In [9]:
dtype_exposure

['raw', 'calexp']

In [10]:
dtype_ingest

['goodSeeingCoadd']

In [ ]:
if any(
        dim in ["exposure", "visit"]
        for dim in [
            d.name for d in registry.queryDatasetTypes(datasetType)[0].dimensions
        ]
    ):
        outside_embargo = [
            dt.id
            for dt in registry.queryDimensionRecords(
                "exposure",
                dataId=dataId,
                datasets=datasetType,
                collections=collections,
                where="NOT exposure.timespan OVERLAPS\
                                                        timespan_embargo",
                bind={"timespan_embargo": timespan_embargo},
            )
        ]

In [67]:
now = astropy.time.Time.now().tai
embargo_hours = 1000.0
embargo_period = astropy.time.TimeDelta(
        embargo_hours * 3600.0, format="sec"
    )
timespan_embargo = Timespan(now - embargo_period, None)
# timespan_embargo = Timespan(now - embargo_period, now)

In [66]:
now

<Time object: scale='tai' format='datetime' value=2023-09-28 19:41:39.378966>

In [7]:
timespan_embargo

Timespan(begin=astropy.time.Time('2023-08-18 02:46:26.072299', scale='tai', format='iso'), end=astropy.time.Time('2023-09-28 18:46:26.072299', scale='tai', format='iso'))

In [8]:
timespan_embargo.begin

<Time object: scale='tai' format='iso' value=2023-08-18 02:46:26.072299>

In [9]:
for i, dt in enumerate(registry.queryDatasets(datasetType=datasetType, collections=collections, 
                                                      where="ingest_date < T'2023-09-28 18:26:02.626054'",
                                                     )):
    print(dt.id)
    if i > 10:
        break

63c693ec-271f-41c6-bb7c-4facdd0751de
f4c285f6-7a5b-42a8-b5ef-9f31a420cb49
15c3e38c-7778-4ca8-9d45-881899e63af8
04e758f1-4b36-4093-ac1d-ee73fc967bbe
13056557-0010-4e69-9c66-ba62298b2cf8
1c0bad05-ed9a-4169-8055-7e4616dff5bd
3fa58bf1-0653-4ddc-bcd4-e1f84df6b1df
74f4fd11-855d-47c6-b087-dd1188b881b4
fd60c3f0-aefa-4810-981b-02231679dfb2
50ee54df-da7a-44fa-8941-46942de8b8a3
3a990fbd-27b3-476c-af59-88f2ff78c59f
c3c3f2a1-b470-4105-b0fe-111e252fd8ca


In [10]:
for i, dt in enumerate(registry.queryDatasets(datasetType=datasetType, collections=collections, 
                                                      where="ingest_date < T'2023-09-28 18:26:02.626054' AND ingest_date > T'2023-08-18 02:45:53.173467'",
                                                     )):
    print(dt.id)
    if i > 10:
        break

63c693ec-271f-41c6-bb7c-4facdd0751de
f4c285f6-7a5b-42a8-b5ef-9f31a420cb49
15c3e38c-7778-4ca8-9d45-881899e63af8
04e758f1-4b36-4093-ac1d-ee73fc967bbe
13056557-0010-4e69-9c66-ba62298b2cf8
1c0bad05-ed9a-4169-8055-7e4616dff5bd
3fa58bf1-0653-4ddc-bcd4-e1f84df6b1df
74f4fd11-855d-47c6-b087-dd1188b881b4
fd60c3f0-aefa-4810-981b-02231679dfb2
50ee54df-da7a-44fa-8941-46942de8b8a3
3a990fbd-27b3-476c-af59-88f2ff78c59f
c3c3f2a1-b470-4105-b0fe-111e252fd8ca


In [68]:
timespan_embargo

Timespan(begin=astropy.time.Time('2023-08-18 03:41:46.374257', scale='tai', format='iso'), end=None)

In [70]:
for i, dt in enumerate(registry.queryDatasets(datasetType=datasetType, collections=collections, 
                                                      where="ingest_date < timespan_embargo_begin",
                                                     bind={"timespan_embargo_begin": timespan_embargo.begin},
                                             )):
    print(dt)
    if i > 10:
        break

In [13]:
for i, dt in enumerate(registry.queryDatasets(datasetType=datasetType, collections=collections, 
                                                      where="ingest_date < timespan_embargo_end AND ingest_date > timespan_embargo_begin",
                                                     bind={"timespan_embargo_begin": timespan_embargo.begin, "timespan_embargo_end": timespan_embargo.end},
                                             )):
    print(dt)
    if i > 10:
        break

goodSeeingCoadd@{band: 'r', skymap: 'latiss_v1', tract: 8188, patch: 66} [sc=ExposureF] (run=LATISS/runs/AUXTEL_DRP_IMAGING_2023-08ABC-07AB-05AB/w_2023_35/PREOPS-3726/20230912T151732Z id=63c693ec-271f-41c6-bb7c-4facdd0751de)
goodSeeingCoadd@{band: 'i', skymap: 'latiss_v1', tract: 8188, patch: 41} [sc=ExposureF] (run=LATISS/runs/AUXTEL_DRP_IMAGING_2023-08ABC-07AB-05AB/w_2023_35/PREOPS-3726/20230912T151732Z id=f4c285f6-7a5b-42a8-b5ef-9f31a420cb49)
goodSeeingCoadd@{band: 'g', skymap: 'latiss_v1', tract: 5839, patch: 8} [sc=ExposureF] (run=LATISS/runs/AUXTEL_DRP_IMAGING_2023-08ABC-07AB-05AB/w_2023_35/PREOPS-3726/20230912T151732Z id=15c3e38c-7778-4ca8-9d45-881899e63af8)
goodSeeingCoadd@{band: 'i', skymap: 'latiss_v1', tract: 8188, patch: 58} [sc=ExposureF] (run=LATISS/runs/AUXTEL_DRP_IMAGING_2023-08ABC-07AB-05AB/w_2023_35/PREOPS-3726/20230912T151732Z id=04e758f1-4b36-4093-ac1d-ee73fc967bbe)
goodSeeingCoadd@{band: 'r', skymap: 'latiss_v1', tract: 5616, patch: 355} [sc=ExposureF] (run=LATISS/

In [ ]:
now = astropy.time.Time.now().tai
embargo_hours = 1000.0
embargo_period = astropy.time.TimeDelta(
        embargo_hours * 3600.0, format="sec"
    )
timespan_embargo = Timespan(now - embargo_period, None)
# timespan_embargo = Timespan(now - embargo_period, now)

In [12]:
for i, dt in enumerate(registry.queryDatasets(datasetType=datasetType, collections=collections, 
                                                      where="ingest_date < timespan_embargo_end AND ingest_date > timespan_embargo_begin",
                                                     bind={"timespan_embargo_begin": timespan_embargo.begin, "timespan_embargo_end": timespan_embargo.end},
                                             )):
    print(dt.id)
    if i > 10:
        break

63c693ec-271f-41c6-bb7c-4facdd0751de
f4c285f6-7a5b-42a8-b5ef-9f31a420cb49
15c3e38c-7778-4ca8-9d45-881899e63af8
04e758f1-4b36-4093-ac1d-ee73fc967bbe
13056557-0010-4e69-9c66-ba62298b2cf8
1c0bad05-ed9a-4169-8055-7e4616dff5bd
3fa58bf1-0653-4ddc-bcd4-e1f84df6b1df
74f4fd11-855d-47c6-b087-dd1188b881b4
fd60c3f0-aefa-4810-981b-02231679dfb2
50ee54df-da7a-44fa-8941-46942de8b8a3
3a990fbd-27b3-476c-af59-88f2ff78c59f
c3c3f2a1-b470-4105-b0fe-111e252fd8ca


In [40]:
# for dt in registry.queryDimensionRecords("exposure",datasets=datasetType,collections=collections,
#             where="exposure.timespan OVERLAPS timespan_embargo",
#             bind={"timespan_embargo": timespan_embargo},
#         ):
#     print(dt.id)
#     if i > 10:
#         break

## check if exposure and vist dimension exist

In [41]:
datasetType = 'goodSeeingCoadd'
collections = 'LATISS/runs/AUXTEL_DRP_IMAGING_2023-08ABC-07AB-05AB/w_2023_35/PREOPS-3726/20230912T151732Z'

In [42]:
for dt in sorted(registry.queryDatasetTypes(datasetType)):
    print(dt)

DatasetType('goodSeeingCoadd', {band, skymap, tract, patch}, ExposureF)


In [48]:
datasetType = 'raw'
collections = 'LATISS/raw/all/'

In [49]:
for dt in sorted(registry.queryDatasetTypes(datasetType)):
    print(dt)

DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure)


In [45]:
# for c in sorted(registry.queryCollections(datasetType='calexp')):
#     print(c)

In [60]:
datasetType = 'calexp'
collections = 'LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616'
for dt in sorted(registry.queryDatasetTypes(datasetType)):
    print(dt)

DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF)


In [55]:
for dt in sorted(registry.queryDatasetTypes(datasetType)):
    print(dt)

DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF)


In [56]:
# for dt in sorted(registry.queryDatasetTypes()):
#     print(dt)

In [57]:
for dt in sorted(registry.queryDatasetTypes('calexp')):
    print(dt)

DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF)


In [58]:
for dt in sorted(registry.queryDatasetTypes('calexp.*')):
    print(dt)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-7.0.1/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3526: FutureWarning: Querying for component datasets via Registry query methods is deprecated in favor of using DatasetRef and DatasetType methods on parent datasets. Only components=False will be supported after v26, and the components argument will be removed after v27.
  exec(code_obj, self.user_global_ns, self.user_ns)


DatasetType('calexp.apCorrMap', {band, instrument, detector, physical_filter, visit}, ApCorr, parentStorageClass=ExposureF)
DatasetType('calexp.bbox', {band, instrument, detector, physical_filter, visit}, Box2I, parentStorageClass=ExposureF)
DatasetType('calexp.coaddInputs', {band, instrument, detector, physical_filter, visit}, CoaddInputs, parentStorageClass=ExposureF)
DatasetType('calexp.detector', {band, instrument, detector, physical_filter, visit}, Detector, parentStorageClass=ExposureF)
DatasetType('calexp.dimensions', {band, instrument, detector, physical_filter, visit}, Extent2I, parentStorageClass=ExposureF)
DatasetType('calexp.filter', {band, instrument, detector, physical_filter, visit}, FilterLabel, parentStorageClass=ExposureF)
DatasetType('calexp.id', {band, instrument, detector, physical_filter, visit}, int, parentStorageClass=ExposureF)
DatasetType('calexp.image', {band, instrument, detector, physical_filter, visit}, ImageF, parentStorageClass=ExposureF)
DatasetType('ca

In [59]:
for i, dt in enumerate(registry.queryDimensionRecords('exposure',datasets=datasetType, collections=collections,
                                             )):
    print(dt)
    if i > 10:
        break

exposure:
  instrument: 'LATISS'
  id: 2022110900640
  physical_filter: 'SDSSg_65mm~empty'
  obs_id: 'AT_O_20221109_000640'
  exposure_time: 30.0
  dark_time: 30.2287
  observation_type: 'science'
  observation_reason: 'object'
  day_obs: 20221109
  seq_num: 640
  seq_start: 640
  seq_end: 641
  group_name: '2022-11-10T04:47:54.142'
  group_id: 2479852741420000
  target_name: 'SPT-CL-J0559-5249_053'
  science_program: 'AUXTEL_DRP_IMAGING'
  tracking_ra: 89.9253975584863
  tracking_dec: -52.63775801969686
  sky_angle: 359.997616534617
  azimuth: 139.4582676183927
  zenith_angle: 36.239877732194145
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-11-10 04:49:23.525011', scale='tai', format='iso'), end=astropy.time.Time('2022-11-10 04:49:53.754000', scale='tai', format='iso'))
exposure:
  instrument: 'LATISS'
  id: 2022110900620
  physical_filter: 'SDSSi_65mm~empty'
  obs_id: 'AT_O_20221109_000620'
  exposure_time: 30.0
  dark_time: 30.2287
  observation_type: 'sc

In [33]:
for i, dt in enumerate(registry.queryDimensionRecords('visit',datasets=datasetType, collections=collections,
                                             )):
    print(dt)
    if i > 10:
        break

visit:
  instrument: 'LATISS'
  id: 2022110900640
  physical_filter: 'SDSSg_65mm~empty'
  name: 'AT_O_20221109_000640'
  day_obs: 20221109
  seq_num: 640
  exposure_time: 30.0
  target_name: 'SPT-CL-J0559-5249_053'
  observation_reason: 'object'
  science_program: 'AUXTEL_DRP_IMAGING'
  azimuth: 139.4582676183927
  zenith_angle: 36.239877732194145
  region: ConvexPolygon([UnitVector3d(0.0018739560247543958, 0.6060174558168964, -0.7954491382445726), UnitVector3d(-0.00024718457727055927, 0.6060203191097887, -0.7954491257936328), UnitVector3d(-0.00024493714049570467, 0.6076796868988159, -0.7941821819559756), UnitVector3d(0.0018762034747673365, 0.6076768235201749, -0.7941821944723714)])
  timespan: Timespan(begin=astropy.time.Time('2022-11-10 04:49:23.525011', scale='tai', format='iso'), end=astropy.time.Time('2022-11-10 04:49:53.754000', scale='tai', format='iso'))
visit:
  instrument: 'LATISS'
  id: 2022110900620
  physical_filter: 'SDSSi_65mm~empty'
  name: 'AT_O_20221109_000620'
  day_

In [ ]:
for i, dt in enumerate(registry.queryDimensionRecords('visit',datasets=='calexp', collections=collections, 
                                                      where="visit.time < timespan_embargo_end AND ingest_date > timespan_embargo_begin",
                                                     bind={"timespan_embargo_begin": timespan_embargo.begin, "timespan_embargo_end": timespan_embargo.end},
                                             )):
    print(dt.id)
    if i > 10:
        break